In [2]:
import os 
from kubernetes import config

class K8sClient():
    def __init__(self) -> None:
        self.in_cluster = None
        
    def connect(self, context: str, pathFile: str):
        self.in_cluster = None
        if self.__is_running_in_k8s():
            self.in_cluster = True
            print("K8sClient connected in cluster")
        else:
            config.load_kube_config(
                config_file = pathFile,
                context = context,
                client_configuration = None,
                persist_config = None 
            )
            self.in_cluster = False
            print(f"K8sClient connected in cluster by {pathFile}")
        
    def __is_running_in_k8s(self):
        return os.path.isdir("/var/run/secrets/kubernetes.io/")
    
client = K8sClient()
client.connect(
    context=os.environ.get("KUBE_CONFIG_DEFAULT_CONTEXT", "minikube"),
    pathFile=os.environ.get("KUBE_CONFIG_DEFAULT_LOCATION", "~/.kube/config")                        
)

K8sClient connected in cluster by ~/.kube/config


In [6]:
from kubernetes import client

# Criar um objeto da API do Kubernetes
api = client.CoreV1Api()

# Obter informações sobre o pod
namespace = "kubeflow-user-example-com"  # Namespace do Katib
pod_name = 'train-api-stop-experiment-l7tjsptg-f4bmb'

pod = api.read_namespaced_pod(pod_name, namespace)

# Listar os nomes dos contêineres no pod
container_names = [c.name for c in pod.spec.containers]
print(container_names)

['training-container', 'istio-proxy', 'metrics-logger-and-collector']


In [5]:
from kubernetes import client

# Criar um objeto da API do Kubernetes
api = client.CoreV1Api()

# Obter informações sobre os pods em execução
namespace = "kubeflow-user-example-com"  # Namespace do Katib
pod_name = 'train-api-stop-experiment-l7tjsptg-92xfd'
container = 'metrics-logger-and-collector'

api_response = api.connect_get_namespaced_pod_exec(
        name=pod_name,
        namespace=namespace,
        container=container,
        command=["wget", "-O-", "http://localhost:5000/"]
    )

# Ler a resposta da API do trial
response_data = api_response.read_stdout()
print(f"Response from trial {pod_name}: {response_data}")


# Iterar sobre os pods e interagir com cada um deles
'''
for pod in pods.items:
    pod_name = pod.metadata.name
    container_name = pod.spec.containers[0].name  # Supondo apenas um contêiner no pod
    
    print(container_name)

    # Enviar uma solicitação para a API do trial
    # Exemplo: Enviar uma solicitação GET para o endpoint "/predict" do contêiner
    
    api_response = api.connect_get_namespaced_pod_exec(
        name=train-api-stop-experiment-l7tjsptg-f4bmb,
        namespace=namespace,
        command=["curl", "http://localhost:5000/"]
    )

    # Ler a resposta da API do trial
    response_data = api_response.read_stdout()
    print(f"Response from trial {pod_name}: {response_data}")
'''

ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Audit-Id': '32d7568e-f9ea-4c36-a5cf-90f37c6430f6', 'Cache-Control': 'no-cache, private', 'Content-Type': 'application/json', 'Date': 'Fri, 26 May 2023 02:23:22 GMT', 'Content-Length': '139'})
HTTP response body: {"kind":"Status","apiVersion":"v1","metadata":{},"status":"Failure","message":"Upgrade request required","reason":"BadRequest","code":400}

